# Project Proposal


In [1]:
library(tidyverse)
library(repr)
library(infer)
library(dplyr)
library(cowplot)
library(broom)
library(rsample)
library(MASS)      
library(car)       

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp



Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some




In [2]:


url <- "https://raw.githubusercontent.com/Arcyill/STAT-301-Project/refs/heads/main/placementdata.csv"

placementData <- read.csv(url)


## Data Description

In [3]:
head(placementData)


,StudentID,CGPA,Internships,Projects,Workshops.Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus
,<int>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<int>,<int>,<chr>
1,1,7.5,1,1,1,65,4.4,No,No,61,79,NotPlaced
2,2,8.9,0,3,2,90,4.0,Yes,Yes,78,82,Placed
3,3,7.3,1,2,2,82,4.8,Yes,No,79,80,NotPlaced
4,4,7.5,1,1,2,85,4.4,Yes,Yes,81,80,Placed
5,5,8.3,1,2,2,86,4.5,Yes,Yes,74,88,Placed
6,6,7.0,0,2,2,71,4.2,Yes,No,55,66,NotPlaced


### Data source (Kaggle): https://www.kaggle.com/datasets/ruchikakumbhar/placement-prediction-dataset

In [4]:
nrow(placementData)

[1] 10000

In [5]:
ncol(placementData)

[1] 12

This dataset contains information about the students' academic and training and placement status. There are 10000 observations and 12 variables consisting of:

- StudentID - The ID of the student ranging from 1 to 10000. (int) 

- CGPA - It is the overall grades achieved by the student. (float)

- Internships - It tells the number of internships a student has done. (Int)

- Projects - Number of projects a student has done. (Int)

- Workshops/Certifications - As there are multiple courses available online student opt for them to upskill themselves. (Int)

- ApptitudeTestScore - Aptitude test are generally a part of the recruitment process to understand the Quant and logical thinking of the student. (Int)

- SoftSkillrating - Communication is a key role that plays in the placement or in any aspect of the life. (float)

- ExtraCurricularActivities - This helps provide and insight about the personality of an individual regarding how much he is active other than the academic. (bool)

- PlacementTraining - It is provided to students in college to ace the placement process. (bool)

- SSC - Senior Secondary Marks. (Int)

- HSC - Higher Secondary Marks. (Int)

- PlacementStatus - This is our target column with two classes "Placed" and not "NotPlaced". (Binary)

### Pre-Selection of Variables

While it may be hard to determine what vairables to drop at this stage, we can drop StudentID because it just a unique identifier for each student, meaning it has no predictive power or meaningful relationship with PlacementStatus. Removing it will prevent unnecessary noise in our analysis and improve model interpretability

In [6]:
placementData <- placementData |> dplyr::select(-StudentID) |> 
        mutate(ExtracurricularActivities = if_else(ExtracurricularActivities == "Yes", 1, 0),               
        PlacementTraining = if_else(PlacementTraining == "Yes", 1, 0),
              PlacementStatus = if_else(PlacementStatus == "Placed", 1, 0))

head(placementData)

,CGPA,Internships,Projects,Workshops.Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus
,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
1,7.5,1,1,1,65,4.4,0,0,61,79,0
2,8.9,0,3,2,90,4.0,1,1,78,82,1
3,7.3,1,2,2,82,4.8,1,0,79,80,0
4,7.5,1,1,2,85,4.4,1,1,81,80,1
5,8.3,1,2,2,86,4.5,1,1,74,88,1
6,7.0,0,2,2,71,4.2,1,0,55,66,0


We will also remove all rows with NA values

In [7]:
placementData <- placementData |> drop_na()
head(placementData)

,CGPA,Internships,Projects,Workshops.Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus
,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
1,7.5,1,1,1,65,4.4,0,0,61,79,0
2,8.9,0,3,2,90,4.0,1,1,78,82,1
3,7.3,1,2,2,82,4.8,1,0,79,80,0
4,7.5,1,1,2,85,4.4,1,1,81,80,1
5,8.3,1,2,2,86,4.5,1,1,74,88,1
6,7.0,0,2,2,71,4.2,1,0,55,66,0


## Question

a. We want to examine the association between a student's likelihood of securing a job placement (PlacementStatus, the response) using various other factors as predictors, including academic achievements (CGPA, SSC_Marks, HSC_Marks), skill enhancement activities (Internships, Projects, Workshops/Certifications), and personal attributes (AptitudeTestScore, SoftSkillsRating, ExtracurricularActivities).

b. The primary focus of this question is to make inferences from how academic achievements, skillenhancement activities, and personal attributes relate to a student’s likelihood of securing a job placement. By using logistic regression, we can interpret the significance and impact of different factors on placement outcomes. As students most likely looking for jobs, instead of focusing on making the most predictions, we are more interested in understanding which variables have meaningful associations with job placement and how they influence the probability of getting placed.

## Exploratory Data Analysis

The stacked histograms illustrate the distribution of students' CGPA, differentiated by Placement Status (using colour) and separated by the number of internships. From the plot, students with higher CGPA tend to secure placements more frequently, especially noticeable among students who, despite having no internships, have more placements than not placed. The plot also reveals that students with two internships show a higher likelihood of placement, even at somewhat lower CGPAs. Conversely, there's little visible distinction between students with zero or one internship in terms of CGPA distribution or placement outcomes. 


There is also a noticeable trend in CGPA distribution based on internship experience. This could be because students who actively pursue internships tend to maintain higher CGPAs overall or vice versa, as evident in the fact that students with 2 internships exhibit a distribution that's left-skewed (higher CGPAs), whereas students with 0 internships exhibit a right-skewed distribution (moderate to lower CGPAs).


This visualization is relevant because it helps us explore the relationship between academic performance, practical experience, and placement success. The plot highlights that CGPA is a key factor in placement but also shows that internships may influence placement likelihood independently of GPA. By faceting the histogram by internship experience, we can observe how CGPA distributions shift based on the number of internships, revealing that students with more internships tend to have both higher CGPAs and higher placement rates. This is crucial for the inference question stated earlier, as it suggests that both academic achievement and internship experience may be important in securing a job placement. 

This is important because it means that excluding either one of these variables could lead to omitted variable bias. For example, if we were to exclude internships from our analysis, our model might overestimate the impact of CGPA on placement success, failing to account for the fact that internships provide an alternative pathway to placement, even for students with moderate CGPAs. On the other hand, omitting CGPA could lead to overstating the effect of internships, ignoring the clear trend that students with higher GPAs are generally more likely to be placed, regardless of internship experience. Therefore, our model should include both CGPA and internships at the very least. 



# Methods and Plan

The model used to address the research question will be logistic regression. This method will model the binary outcome of placement status (placed or not placed based on all other variables as predictors (for the time being). 

Logistic regression is appropriate because placement status is binary. Logistic regression models the probability of placement and allows for the interpretation of how each predictor influences this probability through odd ratios. 

There are several assumptions:
- Observations of placement outcomes for each student must be independent of each other
- Predictors must have a linear relationship with the log odds of the placement outcome.
- Predictor variables are not highly correlated with one another.
- There are no significant outliers that may disproportionately impact the parameter estimates

Potential Limitations or Weakness:
- One significant limitation is the risk of multicollinearity. Since no variable selection was implemented for this stage of the project, if predictors were highly correlated, it could reduce the accuracy and interpretability of the model's coefficients. This could be mitigated by employing a method like LASSO for variable selection. (this will be implemented for the final project)
- Another limitation is that the assumption of linearity between predictors and log odds might not hold, requiring transformations or additional modelling techniques.
- If significant outliers exist, it can significantly affect the parameter estimates.


# Computational Code and Output

The function in R to fit a logistic regression requires either a numerical response (0 and 1) or a `factor`, with two levels (note that R stores factors as integers). 

Mathematically, we have to construct a binary response $Y_i$ that flags the successes for a given event of interest: 

$$
Y_i =
\begin{cases}
1 \; \; \; \; \mbox{if the $i$th observation is Placed},\\
0 \; \; \; \; 	\mbox{if the $i$th observation is NotPlaced}
\end{cases}
$$

Therefore, we will change PlacementStatus with values consisting of 1s and 0s. 

In [9]:

placementDataB <- placementData %>%
  mutate(PlacementStatus = if_else(PlacementStatus == "Placed", 1, 0))
head(placementDataB)


,CGPA,Internships,Projects,Workshops.Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus
,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
1,7.5,1,1,1,65,4.4,0,0,61,79,0
2,8.9,0,3,2,90,4.0,1,1,78,82,0
3,7.3,1,2,2,82,4.8,1,0,79,80,0
4,7.5,1,1,2,85,4.4,1,1,81,80,0
5,8.3,1,2,2,86,4.5,1,1,74,88,0
6,7.0,0,2,2,71,4.2,1,0,55,66,0


We'll estimate a binary logistic regression utilizing the function glm() with PlacementStatus as the response and all others as the predictors. 

In [10]:
placement_model <- tidy(glm(PlacementStatus~., data = placementDataB, family = binomial), exponentiate = TRUE) 
#take the exponential of the coefficients, so we can talk in terms of odds instead of logits.


placement_model

Warning message:
“glm.fit: algorithm did not converge”


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),2.900701e-12,64264.8279,-4.133843e-04,0.9996702
CGPA,1.000000e+00,6819.3103,5.197885e-19,1.0000000
Internships,1.000000e+00,5856.6441,3.625411e-20,1.0000000
Projects,1.000000e+00,5492.9320,-1.701743e-18,1.0000000
Workshops.Certifications,1.000000e+00,4586.6215,9.012058e-19,1.0000000
AptitudeTestScore,1.000000e+00,610.1559,-3.644028e-18,1.0000000
SoftSkillsRating,1.000000e+00,10837.7657,2.006642e-18,1.0000000
ExtracurricularActivities,1.000000e+00,9623.8664,-1.971802e-18,1.0000000
PlacementTraining,1.000000e+00,8760.4169,-1.787773e-18,1.0000000


The logistic regression indicates that there are several predictors that significantly influence whether the student is placed or not placed. Notably, each increase in CGPA increases a student's odds of placement by a factor of 1.47, participating in Placement Training increases placement odds by a factor of 2.52 and involvement in extracurricular activities increases the odds by a factor of 2.11. Conversely, contrary to what the exploratory graph suggested, the number of internships appears to have no meaningful impact on placement likelihood, with an odds ratio of roughly 1.0. One potential problem is that this analysis did not account for possible multicollinearity between predictors, which could be addressed by using LASSO for variable selection.  

In [11]:
set.seed(123) # we set a seed to make our code reproductible 
placementData_split <- initial_split(placementData, prop = 0.7, strata = PlacementStatus)
placementData_training <- training(placementData_split)
placementData_testing <- testing(placementData_split)
head(placementData_training)

,CGPA,Internships,Projects,Workshops.Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus
,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
1,7.5,1,1,1,65,4.4,0,0,61,79,0
2,7.3,1,2,2,82,4.8,1,0,79,80,0
3,7.0,0,2,2,71,4.2,1,0,55,66,0
4,7.7,2,1,0,85,3.5,1,1,59,72,0
5,7.7,1,1,1,74,4.6,0,1,63,73,0
6,7.4,0,2,1,78,4.4,0,0,63,66,0


In [12]:
null_model <- glm(PlacementStatus ~ 1, data = placementData_training, family = 'binomial')
full_model <- glm(PlacementStatus ~ ., data = placementData_training, family = 'binomial')
step_model <- MASS::stepAIC(null_model, scope = list(lower = null_model, upper = full_model), direction = "forward")
summary(step_model)

Start:  AIC=9523.06
PlacementStatus ~ 1

                            Df Deviance    AIC
+ AptitudeTestScore          1   7342.8 7346.8
+ HSC_Marks                  1   7625.4 7629.4
+ ExtracurricularActivities  1   7767.4 7771.4
+ Projects                   1   7823.8 7827.8
+ SSC_Marks                  1   7827.0 7831.0
+ SoftSkillsRating           1   8087.7 8091.7
+ CGPA                       1   8100.7 8104.7
+ Workshops.Certifications   1   8632.3 8636.3
+ PlacementTraining          1   8771.9 8775.9
+ Internships                1   9054.9 9058.9
<none>                           9521.1 9523.1

Step:  AIC=7346.77
PlacementStatus ~ AptitudeTestScore

                            Df Deviance    AIC
+ ExtracurricularActivities  1   6710.2 6716.2
+ SSC_Marks                  1   6840.9 6846.9
+ HSC_Marks                  1   6857.0 6863.0
+ Projects                   1   6921.4 6927.4
+ CGPA                       1   6939.3 6945.3
+ SoftSkillsRating           1   7025.3 7031.3
+ Placeme


Call:
glm(formula = PlacementStatus ~ AptitudeTestScore + ExtracurricularActivities + 
    SSC_Marks + PlacementTraining + CGPA + SoftSkillsRating + 
    Projects + HSC_Marks + Workshops.Certifications, family = "binomial", 
    data = placementData_training)

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -18.252114   0.632117 -28.875  < 2e-16 ***
AptitudeTestScore           0.069434   0.005570  12.465  < 2e-16 ***
ExtracurricularActivities   0.730853   0.080437   9.086  < 2e-16 ***
SSC_Marks                   0.033101   0.003748   8.832  < 2e-16 ***
PlacementTraining           0.887372   0.085136  10.423  < 2e-16 ***
CGPA                        0.458496   0.060208   7.615 2.63e-14 ***
SoftSkillsRating            0.651028   0.100664   6.467 9.97e-11 ***
Projects                    0.272739   0.044543   6.123 9.18e-10 ***
HSC_Marks                   0.023426   0.004575   5.121 3.04e-07 ***
Workshops.Certifications    0.1100

The stepwise AIC selection process helped find the most important factors that influence whether a student gets placed. The final model includes Aptitude Test Score, Extracurricular Activities, SSC Marks, Placement Training, CGPA, Soft Skills Rating, Projects, HSC Marks, and Workshops/Certifications. All of these factors significantly impact placement. Students with higher Aptitude Test Scores, CGPA, and Soft Skills Ratings are more likely to get placed. Participation in extracurricular activities, placement training, and workshops also increases the chances of placement. While SSC and HSC marks play a role, they have a smaller effect compared to other factors.

The biggest influences on placement are Placement Training, Extracurricular Activities, and Aptitude Test Score. This suggests that beyond good grades, practical experience and well-rounded skills are key to getting placed. The model is much better at predicting placement outcomes compared to using no predictors, as shown by the significant drop in AIC (from 9523.06 to 6085.6) and residual deviance (from 9521.1 to 6065.6).

In [17]:
final_model <- glm(PlacementStatus ~ AptitudeTestScore + ExtracurricularActivities + SSC_Marks + PlacementTraining + CGPA + SoftSkillsRating + Projects + HSC_Marks + Workshops.Certifications, data = placementData_testing, family = 'binomial')
summary(final_model)
tidy(final_model)


Call:
glm(formula = PlacementStatus ~ AptitudeTestScore + ExtracurricularActivities + 
    SSC_Marks + PlacementTraining + CGPA + SoftSkillsRating + 
    Projects + HSC_Marks + Workshops.Certifications, family = "binomial", 
    data = placementData_testing)

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -17.138192   0.941935 -18.195  < 2e-16 ***
AptitudeTestScore           0.073760   0.008584   8.593  < 2e-16 ***
ExtracurricularActivities   0.779030   0.122946   6.336 2.35e-10 ***
SSC_Marks                   0.016976   0.005935   2.861 0.004229 ** 
PlacementTraining           1.029746   0.132598   7.766 8.11e-15 ***
CGPA                        0.227427   0.091274   2.492 0.012714 *  
SoftSkillsRating            0.706921   0.154159   4.586 4.53e-06 ***
Projects                    0.237628   0.068754   3.456 0.000548 ***
HSC_Marks                   0.036782   0.006945   5.296 1.18e-07 ***
Workshops.Certifications    0.21797

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-17.13819228,0.941934668,-18.194672,5.687865e-74
AptitudeTestScore,0.07375997,0.008583942,8.592786,8.488583e-18
ExtracurricularActivities,0.77902953,0.122946041,6.336353,2.352673e-10
SSC_Marks,0.01697593,0.005934541,2.860529,4.229348e-03
PlacementTraining,1.02974643,0.132598354,7.765907,8.106325e-15
CGPA,0.22742724,0.091274415,2.491687,1.271382e-02
SoftSkillsRating,0.70692095,0.154159346,4.585651,4.525742e-06
Projects,0.23762822,0.068753569,3.456231,5.477855e-04
HSC_Marks,0.03678209,0.006945024,5.296178,1.182514e-07


In [14]:
# Confusion Matrix
predictions <- predict(final_model, placementData_testing, type = "response")
predicted_classes <- ifelse(predictions > 0.5, 1, 0)
table(Predicted = predicted_classes, Actual = placementData_testing$PlacementStatus)

         Actual
Predicted    0    1
        0 1430  288
        1  311  972

In [16]:
# VIF and ANOVA
vif(final_model)
anova(final_model, test = "Chisq")

AptitudeTestScore ExtracurricularActivities                 SSC_Marks 
                 1.375143                  1.280490                  1.360448 
        PlacementTraining                      CGPA          SoftSkillsRating 
                 1.023019                  1.259211                  1.239669 
                 Projects                 HSC_Marks  Workshops.Certifications 
                 1.350860                  1.405197                  1.159478

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
,<int>,<dbl>,<int>,<dbl>,<dbl>
NULL,NA,NA,3000,4082.841,NA
AptitudeTestScore,1,953.44870,2999,3129.393,2.360925e-209
ExtracurricularActivities,1,281.48686,2998,2847.906,3.561113e-63
SSC_Marks,1,57.23298,2997,2790.673,3.871244e-14
PlacementTraining,1,88.81510,2996,2701.858,4.334950e-21
CGPA,1,24.93614,2995,2676.921,5.926110e-07
SoftSkillsRating,1,36.42587,2994,2640.496,1.585843e-09
Projects,1,21.16790,2993,2619.328,4.207523e-06
HSC_Marks,1,34.23023,2992,2585.097,4.896233e-09


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-17.13819228,0.941934668,-18.194672,5.687865e-74
AptitudeTestScore,0.07375997,0.008583942,8.592786,8.488583e-18
ExtracurricularActivities,0.77902953,0.122946041,6.336353,2.352673e-10
SSC_Marks,0.01697593,0.005934541,2.860529,4.229348e-03
PlacementTraining,1.02974643,0.132598354,7.765907,8.106325e-15
CGPA,0.22742724,0.091274415,2.491687,1.271382e-02
SoftSkillsRating,0.70692095,0.154159346,4.585651,4.525742e-06
Projects,0.23762822,0.068753569,3.456231,5.477855e-04
HSC_Marks,0.03678209,0.006945024,5.296178,1.182514e-07
